In [ ]:
# Install mediapipe if not already installed
#!pip install mediapipe opencv-python

<img src="https://i.imgur.com/3j8BPdc.png" style="height:500px" >

## 📌 What it means:
This measures the **angle between the left shoulder and left ear**.

It indicates how much your **neck is tilted forward or backward**.

---

## 🧠 Intuition:
When sitting upright, your **ear should be vertically aligned above your shoulder**.

If your ear is far forward (like when slouching or looking down), the **angle becomes sharper (smaller)**.

---

## ✅ Good posture:
- `neck_angle` is **high** (e.g., **45°–60° or more**)
- Indicates your **neck is aligned properly**

---

## ❌ Bad posture:
- `neck_angle` is **low** (e.g., **< 25°**)
- Indicates you're likely **leaning your head forward**


# webcam

In [ ]:
import cv2
import math
from mediapipe.python.solutions.pose import POSE_CONNECTIONS
from datetime import datetime

# ✅ MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
font = cv2.FONT_HERSHEY_SIMPLEX

# ✅ Helper functions
def findDistance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def findAngle(x1, y1, x2, y2):
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    return int(180 / math.pi) * theta

def sendWarning():
    print("⚠️ Warning: You've been in bad posture too long!")

# ✅ Parameters
offset_threshold = 100
neck_angle_threshold = 25
torso_angle_threshold = 10
time_threshold = 180  # seconds

good_frames = 0
bad_frames = 0

# ✅ Colors
green = (127, 255, 0)
red = (50, 50, 255)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)
blue = (255, 0, 0)  # ✅ Using blue instead of white

# ✅ Webcam setup
cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width, frame_height = 640, 360

# ✅ Output video file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"webcam_posture_{timestamp}.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

print("🎥 Press 'q' to stop recording...")

try:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("❌ Webcam read failed.")
            break

        image = cv2.resize(image, (frame_width, frame_height))
        h, w = image.shape[:2]
        image = cv2.flip(image, 1)

        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        keypoints = pose.process(rgb_image)

        if keypoints.pose_landmarks:
            lm = keypoints.pose_landmarks.landmark
            lmPose = mp_pose.PoseLandmark

            # ✅ Draw landmarks 0–24 with coordinates
            for i in range(25):
                cx = int(lm[i].x * w)
                cy = int(lm[i].y * h)
                cv2.circle(image, (cx, cy), 4, blue, -1)
                cv2.putText(image, f'{i}', (cx + 5, cy - 5), font, 0.4, pink, 1)

            # ✅ Draw pose skeleton using MediaPipe-defined connections
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx = connection[0]
                end_idx = connection[1]
                x1, y1 = int(lm[start_idx].x * w), int(lm[start_idx].y * h)
                x2, y2 = int(lm[end_idx].x * w), int(lm[end_idx].y * h)
                cv2.line(image, (x1, y1), (x2, y2), blue, 2)

            # ✅ Posture analysis
            l_shldr_x = int(lm[lmPose.LEFT_SHOULDER].x * w)
            l_shldr_y = int(lm[lmPose.LEFT_SHOULDER].y * h)
            r_shldr_x = int(lm[lmPose.RIGHT_SHOULDER].x * w)
            r_shldr_y = int(lm[lmPose.RIGHT_SHOULDER].y * h)
            l_ear_x = int(lm[lmPose.LEFT_EAR].x * w)
            l_ear_y = int(lm[lmPose.LEFT_EAR].y * h)
            l_hip_x = int(lm[lmPose.LEFT_HIP].x * w)
            l_hip_y = int(lm[lmPose.LEFT_HIP].y * h)

            offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

            if offset < offset_threshold:
                cv2.putText(image, f'{int(offset)} Aligned', (w - 250, 30), font, 0.6, green, 2)
            else:
                cv2.putText(image, f'{int(offset)} Not aligned', (w - 250, 30), font, 0.6, red, 2)

            neck_angle = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
            torso_angle = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

            if neck_angle < neck_angle_threshold and torso_angle < torso_angle_threshold:
                bad_frames = 0
                good_frames += 1
                posture = "Good Posture"
                color = light_green
            else:
                good_frames = 0
                bad_frames += 1
                posture = "Bad Posture"
                color = red

            cv2.putText(image, f'Neck: {int(neck_angle)}°', (10, 30), font, 0.6, color, 2)
            cv2.putText(image, f'Torso: {int(torso_angle)}°', (10, 60), font, 0.6, color, 2)
            cv2.putText(image, posture, (10, h - 20), font, 0.8, color, 2)

            # Custom posture lines
            cv2.line(image, (l_shldr_x, l_shldr_y), (l_ear_x, l_ear_y), color, 2)
            cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), color, 2)

            if bad_frames / fps > time_threshold:
                sendWarning()

        # ✅ Save & Show
        video_writer.write(image)
        cv2.imshow("Webcam Posture Tracking", image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("🛑 Exited by user.")
            break

except KeyboardInterrupt:
    print("🛑 Stopped by keyboard interrupt")

# ✅ Cleanup
cap.release()
video_writer.release()
cv2.destroyAllWindows()
print(f"✅ Video saved to: {output_path}")


# video data(testvideo.mp4)

In [ ]:
import cv2
import math
import time
from datetime import datetime
from IPython.display import display, clear_output
from PIL import Image
import mediapipe as mp
from mediapipe.python.solutions.pose import POSE_CONNECTIONS

# ✅ MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
font = cv2.FONT_HERSHEY_SIMPLEX

# ✅ Helper functions
def findDistance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def findAngle(x1, y1, x2, y2):
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1)**2 + (y2 - y1)**2) * y1))
    return int(180 / math.pi) * theta

def sendWarning():
    print("⚠️ Warning: You've been in bad posture too long!")

# ✅ Parameters
offset_threshold = 100
neck_angle_threshold = 25
torso_angle_threshold = 10
time_threshold = 180  # seconds

good_frames = 0
bad_frames = 0

# ✅ Colors
green = (127, 255, 0)
red = (50, 50, 255)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)
white = (255, 255, 255)

# ✅ Load video
input_path = "test.mp4"  # Your video file
cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width, frame_height = 640, 360

# ✅ Setup VideoWriter
output_path = "output_posture.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

try:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("✅ Video complete.")
            break

        image = cv2.resize(image, (frame_width, frame_height))
        h, w = image.shape[:2]
        image = cv2.flip(image, 1)

        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        keypoints = pose.process(rgb_image)

        if keypoints.pose_landmarks:
            lm = keypoints.pose_landmarks.landmark
            lmPose = mp_pose.PoseLandmark

            # ✅ Draw all landmark points (0–24)
            for i in range(0, 25):
                cx = int(lm[i].x * w)
                cy = int(lm[i].y * h)
                cv2.circle(image, (cx, cy), 4, white, -1)
                cv2.putText(image, f'{i}', (cx + 5, cy - 5), font, 0.4, pink, 1)

            # ✅ Connect landmarks using MediaPipe official connections
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx, end_idx = connection
                x1, y1 = int(lm[start_idx].x * w), int(lm[start_idx].y * h)
                x2, y2 = int(lm[end_idx].x * w), int(lm[end_idx].y * h)
                cv2.line(image, (x1, y1), (x2, y2), white, 2)

            # ✅ Posture analysis landmarks
            l_shldr_x = int(lm[lmPose.LEFT_SHOULDER].x * w)
            l_shldr_y = int(lm[lmPose.LEFT_SHOULDER].y * h)
            r_shldr_x = int(lm[lmPose.RIGHT_SHOULDER].x * w)
            r_shldr_y = int(lm[lmPose.RIGHT_SHOULDER].y * h)
            l_ear_x = int(lm[lmPose.LEFT_EAR].x * w)
            l_ear_y = int(lm[lmPose.LEFT_EAR].y * h)
            l_hip_x = int(lm[lmPose.LEFT_HIP].x * w)
            l_hip_y = int(lm[lmPose.LEFT_HIP].y * h)

            offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
            if offset < offset_threshold:
                cv2.putText(image, f'{int(offset)} Aligned', (w - 250, 30), font, 0.6, green, 2)
            else:
                cv2.putText(image, f'{int(offset)} Not aligned', (w - 250, 30), font, 0.6, red, 2)

            neck_angle = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
            torso_angle = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

            if neck_angle < neck_angle_threshold and torso_angle < torso_angle_threshold:
                bad_frames = 0
                good_frames += 1
                posture = "Good Posture"
                color = light_green
            else:
                good_frames = 0
                bad_frames += 1
                posture = "Bad Posture"
                color = red

            # ✅ Text + custom pose lines
            cv2.putText(image, f'Neck: {int(neck_angle)}°', (10, 30), font, 0.6, color, 2)
            cv2.putText(image, f'Torso: {int(torso_angle)}°', (10, 60), font, 0.6, color, 2)
            cv2.putText(image, posture, (10, h - 20), font, 0.8, color, 2)
            cv2.line(image, (l_shldr_x, l_shldr_y), (l_ear_x, l_ear_y), color, 2)
            cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), color, 2)

            if bad_frames / fps > time_threshold:
                sendWarning()

        # ✅ Save and show
        video_writer.write(image)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb_image)
        clear_output(wait=True)
        display(pil_img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("🛑 Stopped manually")
            break

        time.sleep(1 / fps)

except KeyboardInterrupt:
    print("🛑 Interrupted")

# ✅ Cleanup
cap.release()
video_writer.release()
cv2.destroyAllWindows()
print(f"✅ Output video saved to: {output_path}")
